In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#%cd drive/MyDrive/Github/master_thesis
%cd drive/MyDrive/GitHub/master-thesis

/content/drive/MyDrive/GitHub/master-thesis


# Survey Data

In [21]:
df_survey = pd.read_csv('data/220201_preprocessed_data.csv', sep=',',  encoding='utf-8')
df_survey.head()

,gender,postnr,region,Timings_yyyymmdd,Q1_1_feel_exposed,Q1_2_covid_is_threat,Q2_1_easy_to_follow_advice,Q2_2_can_follow_advice_if_wanted,Q2_3_if_follow_advice_safe,Q2_4_if_follow_advice_others_safe,Q2_5_follow_advice_relationships_impared,Q2_6_follow_advice_life_degraded,Q3_1_aware_hand_hygiene,Q3_2_avoid_contact,Q3_3_ensure_frequent_cleaning,Q3_4_avoid_risk_groups,Q3_5_keep_distance,Q3_6_avoid_crowds,Q3_7_minimize_activities_w_contact,Q3b_1_sneeze_sleeve,Q5_1_others_took_distance,Q5_2_others_follow_advice,Q5_3_others_not_care_spreading,Q5_4_yourself_kept_distance,Q5_5_feel_urge_scold,Q6_1_sanctions_are_too_harsh,Q6_2_advices_important,Q6_3_others_can_avoid_spreading,Q6_4_advices_create_fair_burden_dristribution,Q6_5_ownership_of_advice,Q6_6_clear_information_on_advice_reason,Q6_7_advice_limits_daily_activities,Q6_8_advices_enough_for_prevention,Q6_9_trust_political_strategy,Ny1_nr_times_wearing_masks_last_week,Q7_1_last_week_fever,Q7_2_last_week_cough,Q7_3_last_week_sore_throat,Q7_4_last_week_no_smell_taste,Q7_5_last_week_shortness_breath,Q2020NovNy1,Q2020NovNy2_tested_positive,Q2020NovNy3,Q2020NovNy4,Q10_education,Q11_nr_members_household,Q12_nr_children_household,Q4_1_nr_contact_nonhouse_family,Q4_2_nr_contact_colleagues,Q4_3_nr_contact_friends,Q4_4_nr_contact_strangers,age_num,Q14_ocupation_pensioner,Q14_ocupation_employee_private,Q14_ocupation_unemployed,Q14_ocupation_employee_public,Q14_ocupation_self_employed,Q14_ocupation_student,Q14_ocupation_other,Q15_economic_spectrum,Q15_social_spectrum,Q16_economic_spectrum,Q16_social_spectrum,season,timestamp
0,Man,2830,Capital,20210103,2.0,3.0,5.0,7.0,4.0,7.0,4.0,5.0,6.0,7.0,5.0,7.0,6.0,7.0,7.0,7.0,5.0,5.0,5.0,7.0,1.0,1.0,7.0,5.0,2.0,7.0,7.0,6.0,5.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,8,1,0,0,0,0,0,6,0,0,0,1,0,0,0,2.0,4.0,4.0,2.0,4.0,2021-01-03
1,Woman,4654,Zealand,20200731,3.0,7.0,7.0,7.0,5.0,7.0,6.0,5.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,2.0,3.0,2.0,7.0,1.0,2.0,6.0,5.0,4.0,7.0,6.0,2.0,5.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3,1,0,0,0,2,2,5,0,0,0,1,0,0,0,2.0,4.0,2.0,4.0,2.0,2020-07-31
2,Woman,7451,Central,20210605,3.0,7.0,7.0,7.0,7.0,7.0,4.0,3.0,7.0,7.0,5.0,6.0,7.0,7.0,5.0,7.0,4.0,7.0,5.0,7.0,7.0,4.0,7.0,5.0,4.0,4.0,7.0,2.0,4.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4,1,0,5,0,0,0,5,0,0,0,1,0,0,0,2.0,3.0,3.0,4.0,2.0,2021-06-05
3,Woman,6740,Southern,20200826,2.0,2.0,3.0,3.0,7.0,7.0,1.0,1.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,3.0,5.0,3.0,6.0,3.0,3.0,7.0,4.0,5.0,7.0,7.0,7.0,7.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1,1,0,0,0,0,0,5,0,0,0,1,0,0,0,2.0,4.0,3.0,3.0,2.0,2020-08-26
4,Man,4700,Zealand,20201106,6.0,6.0,1.0,7.0,2.0,1.0,7.0,4.0,7.0,3.0,7.0,7.0,7.0,7.0,5.0,7.0,2.0,5.0,3.0,7.0,7.0,4.0,7.0,4.0,1.0,1.0,7.0,3.0,7.0,7.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3,1,0,4,1,4,15,5,1,0,0,0,0,0,0,3.0,4.0,2.0,4.0,3.0,2020-11-06


In [22]:
non_behaviour_cols = ['gender','postnr', 'region', 'Timings_yyyymmdd', 'season','Q10_education', 'Q11_nr_members_household',
 'Q12_nr_children_household','age_num', 'Q14_ocupation_pensioner',
 'Q14_ocupation_employee_private',
 'Q14_ocupation_unemployed',
 'Q14_ocupation_employee_public',
 'Q14_ocupation_self_employed',
 'Q14_ocupation_student',
 'Q14_ocupation_other',
 'Q15_economic_spectrum',
 'Q15_social_spectrum',
 'Q16_economic_spectrum',
 'Q16_social_spectrum',
 'Q7_1_last_week_fever',
 'Q7_2_last_week_cough',
 'Q7_3_last_week_sore_throat',
 'Q7_4_last_week_no_smell_taste',
 'Q7_5_last_week_shortness_breath',
 'Q2020NovNy1',
 'Q2020NovNy2_tested_positive',
 'Q2020NovNy3',
 'Q2020NovNy4']

In [24]:
behaviour_cols = df_survey.columns.tolist()
for element in non_behaviour_cols:
    if element in behaviour_cols:
        behaviour_cols.remove(element)

df_survey_behaviour = df_survey[behaviour_cols]
df_survey_behaviour.head()

,Q1_1_feel_exposed,Q1_2_covid_is_threat,Q2_1_easy_to_follow_advice,Q2_2_can_follow_advice_if_wanted,Q2_3_if_follow_advice_safe,Q2_4_if_follow_advice_others_safe,Q2_5_follow_advice_relationships_impared,Q2_6_follow_advice_life_degraded,Q3_1_aware_hand_hygiene,Q3_2_avoid_contact,Q3_3_ensure_frequent_cleaning,Q3_4_avoid_risk_groups,Q3_5_keep_distance,Q3_6_avoid_crowds,Q3_7_minimize_activities_w_contact,Q3b_1_sneeze_sleeve,Q5_1_others_took_distance,Q5_2_others_follow_advice,Q5_3_others_not_care_spreading,Q5_4_yourself_kept_distance,Q5_5_feel_urge_scold,Q6_1_sanctions_are_too_harsh,Q6_2_advices_important,Q6_3_others_can_avoid_spreading,Q6_4_advices_create_fair_burden_dristribution,Q6_5_ownership_of_advice,Q6_6_clear_information_on_advice_reason,Q6_7_advice_limits_daily_activities,Q6_8_advices_enough_for_prevention,Q6_9_trust_political_strategy,Ny1_nr_times_wearing_masks_last_week,Q4_1_nr_contact_nonhouse_family,Q4_2_nr_contact_colleagues,Q4_3_nr_contact_friends,Q4_4_nr_contact_strangers,timestamp
0,2.0,3.0,5.0,7.0,4.0,7.0,4.0,5.0,6.0,7.0,5.0,7.0,6.0,7.0,7.0,7.0,5.0,5.0,5.0,7.0,1.0,1.0,7.0,5.0,2.0,7.0,7.0,6.0,5.0,6.0,4.0,0,0,0,0,2021-01-03
1,3.0,7.0,7.0,7.0,5.0,7.0,6.0,5.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,2.0,3.0,2.0,7.0,1.0,2.0,6.0,5.0,4.0,7.0,6.0,2.0,5.0,7.0,0.0,0,0,2,2,2020-07-31
2,3.0,7.0,7.0,7.0,7.0,7.0,4.0,3.0,7.0,7.0,5.0,6.0,7.0,7.0,5.0,7.0,4.0,7.0,5.0,7.0,7.0,4.0,7.0,5.0,4.0,4.0,7.0,2.0,4.0,7.0,1.0,5,0,0,0,2021-06-05
3,2.0,2.0,3.0,3.0,7.0,7.0,1.0,1.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,3.0,5.0,3.0,6.0,3.0,3.0,7.0,4.0,5.0,7.0,7.0,7.0,7.0,7.0,1.0,0,0,0,0,2020-08-26
4,6.0,6.0,1.0,7.0,2.0,1.0,7.0,4.0,7.0,3.0,7.0,7.0,7.0,7.0,5.0,7.0,2.0,5.0,3.0,7.0,7.0,4.0,7.0,4.0,1.0,1.0,7.0,3.0,7.0,7.0,4.0,4,1,4,15,2020-11-06


In [25]:
df_survey_behaviour.describe()

,Q1_1_feel_exposed,Q1_2_covid_is_threat,Q2_1_easy_to_follow_advice,Q2_2_can_follow_advice_if_wanted,Q2_3_if_follow_advice_safe,Q2_4_if_follow_advice_others_safe,Q2_5_follow_advice_relationships_impared,Q2_6_follow_advice_life_degraded,Q3_1_aware_hand_hygiene,Q3_2_avoid_contact,Q3_3_ensure_frequent_cleaning,Q3_4_avoid_risk_groups,Q3_5_keep_distance,Q3_6_avoid_crowds,Q3_7_minimize_activities_w_contact,Q3b_1_sneeze_sleeve,Q5_1_others_took_distance,Q5_2_others_follow_advice,Q5_3_others_not_care_spreading,Q5_4_yourself_kept_distance,Q5_5_feel_urge_scold,Q6_1_sanctions_are_too_harsh,Q6_2_advices_important,Q6_3_others_can_avoid_spreading,Q6_4_advices_create_fair_burden_dristribution,Q6_5_ownership_of_advice,Q6_6_clear_information_on_advice_reason,Q6_7_advice_limits_daily_activities,Q6_8_advices_enough_for_prevention,Q6_9_trust_political_strategy,Ny1_nr_times_wearing_masks_last_week,Q4_1_nr_contact_nonhouse_family,Q4_2_nr_contact_colleagues,Q4_3_nr_contact_friends,Q4_4_nr_contact_strangers
count,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000,125301.000000
mean,3.112505,4.797959,5.461393,5.983097,5.425671,6.031596,4.287739,3.468248,6.302998,5.521177,5.156934,6.025467,5.627561,5.480690,5.188953,6.457650,4.290277,4.754431,3.549341,5.839921,2.678550,3.256327,5.876178,4.521879,4.422790,5.614041,5.273062,4.220445,5.081923,5.026983,1.923855,1.573802,1.644440,1.836107,2.812859
std,1.806544,1.775548,1.685913,1.618918,1.686179,1.609975,2.008128,1.920862,1.160594,1.668814,1.684453,1.468579,1.586235,1.737821,1.789252,1.101854,1.633332,1.463266,1.605065,1.327309,2.031489,1.802975,1.485132,1.560264,1.602313,1.578223,1.761842,1.944996,1.586103,1.840642,1.469668,2.236815,2.951331,2.860785,5.548347
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,4.000000,5.000000,6.000000,4.000000,6.000000,3.000000,2.000000,6.000000,5.000000,4.000000,5.000000,5.000000,5.000000,4.000000,6.000000,3.000000,4.000000,2.000000,5.000000,1.000000,2.000000,5.000000,4.000000,4.000000,5.000000,4.000000,3.000000,4.000000,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,5.000000,6.000000,7.000000,6.000000,7.000000,4.000000,4.000000,7.000000,6.000000,5.000000,7.000000,6.000000,6.000000,6.000000,7.000000,4.000000,5.000000,4.000000,6.000000,2.000000,3.000000,6.000000,5.000000,4.000000,6.000000,6.000000,4.000000,5.000000,5.000000,2.000000,0.000000,0.000000,0.000000,0.000000
75%,4.000000,7.000000,7.000000,7.000000,7.000000,7.000000,6.000000,5.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,5.000000,6.000000,5.000000,7.000000,4.000000,4.000000,7.000000,6.000000,5.000000,7.000000,7.000000,6.000000,6.000000,7.000000,3.000000,2.000000,2.000000,2.000000,3.000000
max,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,4.000000,8.000000,10.000000,10.000000,20.000000


In [38]:
df_survey_rolavg = df_survey_behaviour.groupby('timestamp').mean().rolling(7).mean().reset_index()
df_survey_rolavg.dropna(inplace=True)
df_survey_rolavg.head()

,timestamp,Q1_1_feel_exposed,Q1_2_covid_is_threat,Q2_1_easy_to_follow_advice,Q2_2_can_follow_advice_if_wanted,Q2_3_if_follow_advice_safe,Q2_4_if_follow_advice_others_safe,Q2_5_follow_advice_relationships_impared,Q2_6_follow_advice_life_degraded,Q3_1_aware_hand_hygiene,Q3_2_avoid_contact,Q3_3_ensure_frequent_cleaning,Q3_4_avoid_risk_groups,Q3_5_keep_distance,Q3_6_avoid_crowds,Q3_7_minimize_activities_w_contact,Q3b_1_sneeze_sleeve,Q5_1_others_took_distance,Q5_2_others_follow_advice,Q5_3_others_not_care_spreading,Q5_4_yourself_kept_distance,Q5_5_feel_urge_scold,Q6_1_sanctions_are_too_harsh,Q6_2_advices_important,Q6_3_others_can_avoid_spreading,Q6_4_advices_create_fair_burden_dristribution,Q6_5_ownership_of_advice,Q6_6_clear_information_on_advice_reason,Q6_7_advice_limits_daily_activities,Q6_8_advices_enough_for_prevention,Q6_9_trust_political_strategy,Ny1_nr_times_wearing_masks_last_week,Q4_1_nr_contact_nonhouse_family,Q4_2_nr_contact_colleagues,Q4_3_nr_contact_friends,Q4_4_nr_contact_strangers
6,2020-06-04,2.995549,4.453157,5.428551,5.991765,5.552845,6.142242,4.198462,3.334155,6.292879,5.690822,5.165475,6.268608,5.774064,5.647043,5.336773,6.513225,4.404382,4.786330,3.609047,5.958682,2.811251,3.263696,5.914927,4.650648,4.476556,5.068771,5.340096,4.282894,5.495752,5.153057,2.008259,1.683018,1.561180,1.670220,2.392491
7,2020-06-05,3.022097,4.495730,5.452471,5.982480,5.590853,6.133009,4.190758,3.320131,6.320386,5.727854,5.151311,6.253468,5.775041,5.648462,5.316476,6.522394,4.412830,4.783185,3.613152,5.972898,2.771598,3.232490,5.936765,4.624850,4.496307,5.091424,5.361551,4.246363,5.482012,5.211363,1.960052,1.585094,1.593548,1.757952,2.438459
8,2020-06-06,3.024073,4.479826,5.442718,5.979854,5.584992,6.120343,4.167071,3.304084,6.298705,5.683466,5.132054,6.236499,5.744003,5.613060,5.286608,6.511189,4.417907,4.778695,3.598887,5.936982,2.722467,3.235904,5.952306,4.619536,4.500494,5.086128,5.367749,4.229852,5.478537,5.198839,1.948058,1.662631,1.505364,1.751326,2.441005
9,2020-06-07,3.028295,4.481273,5.476273,5.992447,5.594914,6.110554,4.187130,3.293970,6.306551,5.691892,5.154639,6.212960,5.747913,5.630270,5.281339,6.507467,4.423917,4.802533,3.583156,5.952578,2.724095,3.218308,5.966646,4.647790,4.533394,5.129071,5.406505,4.202666,5.486523,5.212708,1.929035,1.646057,1.473094,1.681120,2.484238
10,2020-06-08,3.036817,4.462824,5.498512,5.990825,5.610607,6.111388,4.209969,3.297362,6.326094,5.697479,5.197066,6.233500,5.766615,5.633040,5.301725,6.506340,4.439598,4.814526,3.544330,5.955900,2.715652,3.225227,5.997463,4.661222,4.558453,5.148739,5.423095,4.226457,5.486105,5.247502,1.936010,1.593363,1.595156,1.645763,2.458951


# R-Value

In [39]:
df_obs = pd.read_csv('data/observations_03022022.csv', sep=';',  encoding='utf-8')
df_obs.head()

,Date,Number_of_deaths,Cumulative_number_of_deaths,Confirmed_cases,Admissions_hospital,7day_r_value
0,2020-02-26,0,0,1,0,NaN
1,2020-02-27,0,0,1,0,NaN
2,2020-02-28,0,0,1,0,NaN
3,2020-03-01,0,0,1,1,NaN
4,2020-03-02,0,0,4,0,NaN


# Combine in one df

In [40]:
df = pd.merge(left=df_survey_rolavg, right=df_obs, left_on='timestamp', right_on='Date', how='inner')
df.head()

,timestamp,Q1_1_feel_exposed,Q1_2_covid_is_threat,Q2_1_easy_to_follow_advice,Q2_2_can_follow_advice_if_wanted,Q2_3_if_follow_advice_safe,Q2_4_if_follow_advice_others_safe,Q2_5_follow_advice_relationships_impared,Q2_6_follow_advice_life_degraded,Q3_1_aware_hand_hygiene,Q3_2_avoid_contact,Q3_3_ensure_frequent_cleaning,Q3_4_avoid_risk_groups,Q3_5_keep_distance,Q3_6_avoid_crowds,Q3_7_minimize_activities_w_contact,Q3b_1_sneeze_sleeve,Q5_1_others_took_distance,Q5_2_others_follow_advice,Q5_3_others_not_care_spreading,Q5_4_yourself_kept_distance,Q5_5_feel_urge_scold,Q6_1_sanctions_are_too_harsh,Q6_2_advices_important,Q6_3_others_can_avoid_spreading,Q6_4_advices_create_fair_burden_dristribution,Q6_5_ownership_of_advice,Q6_6_clear_information_on_advice_reason,Q6_7_advice_limits_daily_activities,Q6_8_advices_enough_for_prevention,Q6_9_trust_political_strategy,Ny1_nr_times_wearing_masks_last_week,Q4_1_nr_contact_nonhouse_family,Q4_2_nr_contact_colleagues,Q4_3_nr_contact_friends,Q4_4_nr_contact_strangers,Date,Number_of_deaths,Cumulative_number_of_deaths,Confirmed_cases,Admissions_hospital,7day_r_value
0,2020-06-04,2.995549,4.453157,5.428551,5.991765,5.552845,6.142242,4.198462,3.334155,6.292879,5.690822,5.165475,6.268608,5.774064,5.647043,5.336773,6.513225,4.404382,4.786330,3.609047,5.958682,2.811251,3.263696,5.914927,4.650648,4.476556,5.068771,5.340096,4.282894,5.495752,5.153057,2.008259,1.683018,1.561180,1.670220,2.392491,2020-06-04,4,586,41,5,0.961131
1,2020-06-05,3.022097,4.495730,5.452471,5.982480,5.590853,6.133009,4.190758,3.320131,6.320386,5.727854,5.151311,6.253468,5.775041,5.648462,5.316476,6.522394,4.412830,4.783185,3.613152,5.972898,2.771598,3.232490,5.936765,4.624850,4.496307,5.091424,5.361551,4.246363,5.482012,5.211363,1.960052,1.585094,1.593548,1.757952,2.438459,2020-06-05,2,588,34,9,0.920290
2,2020-06-06,3.024073,4.479826,5.442718,5.979854,5.584992,6.120343,4.167071,3.304084,6.298705,5.683466,5.132054,6.236499,5.744003,5.613060,5.286608,6.511189,4.417907,4.778695,3.598887,5.936982,2.722467,3.235904,5.952306,4.619536,4.500494,5.086128,5.367749,4.229852,5.478537,5.198839,1.948058,1.662631,1.505364,1.751326,2.441005,2020-06-06,3,591,15,2,0.891304
3,2020-06-07,3.028295,4.481273,5.476273,5.992447,5.594914,6.110554,4.187130,3.293970,6.306551,5.691892,5.154639,6.212960,5.747913,5.630270,5.281339,6.507467,4.423917,4.802533,3.583156,5.952578,2.724095,3.218308,5.966646,4.647790,4.533394,5.129071,5.406505,4.202666,5.486523,5.212708,1.929035,1.646057,1.473094,1.681120,2.484238,2020-06-07,2,593,14,4,0.907143
4,2020-06-08,3.036817,4.462824,5.498512,5.990825,5.610607,6.111388,4.209969,3.297362,6.326094,5.697479,5.197066,6.233500,5.766615,5.633040,5.301725,6.506340,4.439598,4.814526,3.544330,5.955900,2.715652,3.225227,5.997463,4.661222,4.558453,5.148739,5.423095,4.226457,5.486105,5.247502,1.936010,1.593363,1.595156,1.645763,2.458951,2020-06-08,0,593,43,5,0.834559


In [41]:
corr = df.corr()

In [46]:
corr['7day_r_value'].sort_values()

Ny1_nr_times_wearing_masks_last_week            -0.289239
Q6_3_others_can_avoid_spreading                 -0.285181
Number_of_deaths                                -0.267957
Q6_7_advice_limits_daily_activities             -0.265305
Q5_2_others_follow_advice                       -0.248026
Q5_1_others_took_distance                       -0.234130
Q3_7_minimize_activities_w_contact              -0.231655
Q5_4_yourself_kept_distance                     -0.206697
Q2_5_follow_advice_relationships_impared        -0.205429
Q3_6_avoid_crowds                               -0.202583
Q3_5_keep_distance                              -0.201502
Q3_2_avoid_contact                              -0.179492
Q2_6_follow_advice_life_degraded                -0.178610
Q3_4_avoid_risk_groups                          -0.165883
Admissions_hospital                             -0.145666
Cumulative_number_of_deaths                     -0.136211
Q2_3_if_follow_advice_safe                      -0.134564
Q3_3_ensure_fr

In [42]:
import plotly.express as px

fig = px.imshow(corr, width=1100, height=1100, color_continuous_scale='RdBu_r', range_color=[-1,1])
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = np.arange(62),
        ticktext = corr.columns.values),
    yaxis = dict(
        tickmode = 'array',
        tickvals = np.arange(62),
        ticktext = corr.columns.values
    )
)
fig.show()